In [1]:
from fuvtds_base_class import FUVTDSBase
from fuvtds_base_class import FUVTDSMonitor

In [2]:
x = FUVTDSBase(PIDs='fuvtds_analysis_list.dat')

  0%|          | 0/1766 [00:00<?, ?it/s]

+++ Scaling LP4 to LP3 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv20eyq_x1d.fits.gz /grp/hst/cos2/cosmo/14854/ldcv22wxq_x1d.fits.gz
+++ Scaling LP3 to LP4 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv22wxq_x1d.fits.gz /grp/hst/cos2/cosmo/14854/ldcv20eyq_x1d.fits.gz
+++ Scaling LP3 to LP2 using data from datasets: /grp/hst/cos2/cosmo/13967/lcqz03osq_x1d.fits.gz /grp/hst/cos2/cosmo/13967/lcqz04iiq_x1d.fits.gz
+++ Scaling LP2 to LP1 using data from datasets: /grp/hst/cos2/cosmo/12806/lbxmt1nxq_x1d.fits.gz /grp/hst/cos2/cosmo/12806/lbxm1bvsq_x1d.fits.gz
+++ Scaling LP4 to LP3 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv20elq_x1d.fits.gz /grp/hst/cos2/cosmo/14854/ldcv22wpq_x1d.fits.gz
+++ Scaling LP3 to LP2 using data from datasets: /grp/hst/cos2/cosmo/13967/lcqz03ogq_x1d.fits.gz /grp/hst/cos2/cosmo/13967/lcqz04h8q_x1d.fits.gz
+++ Scaling LP6 to LP4 using data from datasets: /grp/hst/cos2/cosmo/16830/ler15akzq_x1d.fits.gz /grp/hst/cos2/cosmo/16830/ler15am

IndexingError: Too many indexers